### Import reviews

In [1]:
%load_ext autoreload
%autoreload 2

from src.utils import log_progress as lp

In [2]:
import pickle

DATA_FILE1 = 'data/data.pkl'
DATA_FILE2 = 'data/labels.pkl'
DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

with open(DATA_FILE1, 'rb') as f:
    reviews = pickle.load(f)
    
with open(DATA_FILE2, 'rb') as f:
    stars = pickle.load(f)

with open(DATA_FILE3, 'rb') as f:
    pos_adjectives_list = pickle.load(f)
    
with open(DATA_FILE4, 'rb') as f:
    neu_adjectives_list = pickle.load(f)

with open(DATA_FILE5, 'rb') as f:
    neg_adjectives_list = pickle.load(f)

In [ ]:
print(pos_adjectives_list)

In [3]:
from babble.babbler import BabbleStream

babbler = BabbleStream(reviews,stars,balanced=True, shuffled=True, seed=456)

Grammar construction complete.


In [ ]:
babbler.semparser.grammar.lexical_rules

In [15]:
from babble.utils import display_candidate

candidate = babbler.next()
print(candidate)
print(candidate.text)
print(candidate.entity_positions)

RelationMention(doc_id=1174: entities=("pizza"(9:14), "caesar"(20:26), "salad"(27:32), "service"(73:80), "good"(85:89), "got"(97:100))
mediocre pizza, bad caesar salad, and high prices. on the positive side, service was good and we got our order very quickly.
[(9, 14), (20, 26), (27, 32), (73, 80), (85, 89), (97, 100)]


In [ ]:
print(dir(candidate.entities[0]))
print(candidate.entities)
print(candidate.entities.__getitem__(0))

In [ ]:
def check(candidate,adjective):
    for entity in candidate.entities:
        # qui controllo se gli aggettvi sono vicini alle entità...da capire bene come fare
        print(entity.entity)

In [ ]:
check(candidate,'fsdfsd')

In [16]:
'''
praticamente posso fare cosi: per ogni entità da X a Q, verifico se ci sono aggettivi appena prima o appena dopo(
da capire se può creare problemi") e creo in modo automatico la explanation.
Forse è il caso di provare prima con il solo check dell'aggettivo appena prima"
'''
from babble import Explanation
explanation = Explanation(
        name='LF_1',
        label=1,
        condition='the word "mediocre" is before X or the word "bad" is before Y or the word "good" is after T',
        candidate=candidate
    )

In [18]:
babbler.view_parse(parses[0])

Name: LF_1_4
Parse: return 1 if ('mediocre'.in(text(greater than 0 word(s) to the left of X)) or ('bad'.(= text(Y)) or 'good'.(= text(T)))) else 0


In [ ]:
print(parses[0].semantics)

In [17]:
parses, filtered = babbler.apply(explanation)

/ |#                                                  | 0 Elapsed Time: 0:00:00

Building list of target candidate ids...
All 1 explanations are already linked to candidates.


| |   #                                               | 0 Elapsed Time: 0:00:00


1 explanation(s) out of 1 were parseable.
10 parse(s) generated from 1 explanation(s).
10 parse(s) remain (0 parse(s) removed by DuplicateSemanticsFilter).
8 parse(s) remain (2 parse(s) removed by ConsistencyFilter).
Applying labeling functions to investigate labeling signature.
[========================================] 100%

8 parse(s) remain (0 parse(s) removed by UniformSignatureFilter: (0 None, 0 All)).
4 parse(s) remain (4 parse(s) removed by DuplicateSignatureFilter).
1 parse(s) remain (3 parse(s) removed by LowestCoverageFilter).


In [ ]:
def create_condition(words):
    condition = 'the word '
    for word in words:
        condition = condition + '"'+ word + '" or '
    return condition + 'are in the sentence'

In [ ]:
import spacy

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')

In [ ]:
from babble import Explanation

final_parses = []

index = 0

for element,label,selected_words in (zip(lp.log_progress(reviews[0]),stars[0],tokens_list)):
    doc = spacy_nlp(element.text)
    tokens = [token.text for token in doc if not token.is_stop and not token.pos_  in ('PUNCT', 'NUM', 'SYM')]
        
    print(selected_words)
    
    explanation = Explanation(
        name='LF_' + str(index),
        label=int(label),
        condition=create_condition(selected_words),
        candidate=element
    )
    parses, filtered = babbler.apply(explanation)
    babbler.commit()
    
    index = index + 1

In [ ]:
from metal.analysis import lf_summary

Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], stars[1], lf_names=lf_names)

In [ ]:
from babble.utils import ExplanationIO

explanations = babbler.get_explanations()

FILE = "my_explanations.tsv"
exp_io = ExplanationIO()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)